<a href="https://colab.research.google.com/github/holictoweb/spark_deep_dive/blob/master/spark_table/spark_table_statistics_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit
 
db_path = '/content/drive/MyDrive/data-warehouse/stocklab_db'

spark = SparkSession.builder.appName('test_spark') \
  .config("spark.sql.warehouse.dir", db_path) \
  .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
  .enableHiveSupport() \
  .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

#spark.conf.set("spark.executor.memory", '1g')
#spark.conf.set('spark.executor.cores', '1')
#spark.conf.set('spark.cores.max', '1')
#spark.conf.set("spark.driver.memory",'1g')
sc = spark.sparkContext
sc.getConf().getAll()

[('spark.jars',
  'file:///root/.ivy2/jars/io.delta_delta-core_2.12-0.8.0.jar,file:///root/.ivy2/jars/org.antlr_antlr4-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr4-runtime-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr-runtime-3.5.2.jar,file:///root/.ivy2/jars/org.antlr_ST4-4.0.8.jar,file:///root/.ivy2/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar,file:///root/.ivy2/jars/org.glassfish_javax.json-1.0.4.jar,file:///root/.ivy2/jars/com.ibm.icu_icu4j-58.2.jar'),
 ('spark.driver.host', 'b94351e17554'),
 ('spark.driver.port', '39703'),
 ('spark.app.initial.jar.urls',
  'spark://b94351e17554:39703/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar,spark://b94351e17554:39703/jars/org.antlr_antlr4-runtime-4.7.jar,spark://b94351e17554:39703/jars/org.antlr_antlr-runtime-3.5.2.jar,spark://b94351e17554:39703/jars/org.antlr_antlr4-4.7.jar,spark://b94351e17554:39703/jars/org.glassfish_javax.json-1.0.4.jar,spark://b94351e17554:39703/jars/io.delta_delta-core_2.12-0.8.0.j

In [ ]:
print(spark.conf.get("spark.sql.inMemoryColumnarStorage.compressed"))

print(spark.conf.get("spark.sql.adaptive.enabled"))
print(spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled"))
print(spark.conf.get("spark.sql.adaptive.coalescePartitions.minPartitionNum"))

print(spark.conf.get("spark.sql.hive.metastore.jars"))


true
false
true
None
builtin


In [ ]:
#spark.sql("drop database if exists stocklab CASCADE")
#AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: InvalidOperationException(message:Database stocklab is not empty. One or more tables exist.)
#force drop cascade
spark.sql("create database IF NOT EXISTS stocklab")
spark.sql("use stocklab")
spark.sql("show databases ").show()
spark.sql("describe database stocklab").show(100,False)
spark.sql("show tables").show()

+---------+
|namespace|
+---------+
|  default|
| stocklab|
+---------+

+-------------+------------------------------------------------------------------+
|info_name    |info_value                                                        |
+-------------+------------------------------------------------------------------+
|Database Name|stocklab                                                          |
|Comment      |                                                                  |
|Location     |file:/content/drive/MyDrive/data-warehouse/stocklab_db/stocklab.db|
|Owner        |root                                                              |
+-------------+------------------------------------------------------------------+

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [ ]:
df_day = spark.read.format('parquet').load('/content/drive/MyDrive/data-warehouse/stock_day/Code=005930')
#df_day.write.saveAsTable('stocklab.stock_day')
#df_day.write.format("delta").saveAsTable("stocklab.delta_stock_day")
df_day.createOrReplaceTempView("view_stock_day")

In [ ]:
spark.sql("show tables").show()
spark.sql("DESCRIBE  EXTENDED delta_stock_day ").show(truncate = False)
spark.sql("ANALYZE TABLE delta_stock_day COMPUTE STATISTICS NOSCAN").show(truncate = False)



+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|stocklab|delta_stock_day|      false|
|stocklab|      stock_day|      false|
+--------+---------------+-----------+

+----------------------------+----------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                         |comment|
+----------------------------+----------------------------------------------------------------------------------+-------+
|Open                        |double                                                                            |       |
|High                        |double                                                                            |       |
|Low                         |double                                                                            |       |
|Close                       |dou

AnalysisException: ignored

## Analyze table and columns
https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-aux-describe-table.html


- temp view에는 적용이 불가능.
- 

In [ ]:
spark.sql("DESCRIBE  EXTENDED stock_day ").show(truncate = False)
spark.sql("ANALYZE TABLE stock_day COMPUTE STATISTICS NOSCAN").show(truncate = False)
spark.sql("DESCRIBE  EXTENDED stock_day ").show(truncate = False)
spark.sql("ANALYZE TABLE stock_day COMPUTE STATISTICS").show(truncate = False)
spark.sql("DESCRIBE  EXTENDED stock_day ").show(truncate = False)
spark.sql("ANALYZE TABLE stock_day COMPUTE STATISTICS FOR COLUMNS code").show(truncate = False)
# ANALYZE TABLE table_name COMPUTE STATISTICS FOR COLUMNS column_name [column_name,
spark.sql("DESCRIBE  EXTENDED stock_day code ").show(truncate = False)






+----------------------------+----------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                   |comment|
+----------------------------+----------------------------------------------------------------------------+-------+
|Open                        |double                                                                      |null   |
|High                        |double                                                                      |null   |
|Low                         |double                                                                      |null   |
|Close                       |double                                                                      |null   |
|Volume                      |double                                                                      |null   |
|Code                        |string                                    

In [ ]:
#spark.sql("ANALYZE TABLE stock_day COMPUTE STATISTICS FOR ALL COLUMNS")
spark.sql("DESCRIBE  EXTENDED stock_day ").show(truncate = False)

+----------------------------+----------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                   |comment|
+----------------------------+----------------------------------------------------------------------------+-------+
|Open                        |double                                                                      |null   |
|High                        |double                                                                      |null   |
|Low                         |double                                                                      |null   |
|Close                       |double                                                                      |null   |
|Volume                      |double                                                                      |null   |
|Code                        |string                                    

## spark statistics 성능 비교 

관련한 성능 

## pyspark 사용방법
- pip install pyspark 설치 이후 
  stored in directory 위치 확인 
  
/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

- spark home 지정

PYSPARK_PYTHON=python3 SPARK_HOME=~/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

In [ ]:
# 기본 설정 

!pip install pyspark
!pip install yfinance

#!PYSPARK_PYTHON=python3 SPARK_HOME=~/root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f

from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/data-warehouse/stock_day

#4/1AY0e-g4gHCUKl2LbLgrZScXnG9-_e0Pge08syfCh0dnVO_4oAW7VLqGDMYc

     |████████████████████████████████| 212.4MB 53kB/s 
     |████████████████████████████████| 204kB 16.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=494f1eab581761ae6943499e73fdc9de3f8eafeb9a153be7ff43d56e2e37e894
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
     |████████████████████████████████| 6.3MB 2.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23455 sha256=3043c1ba15aa12b16ebcb02267180ed40bc7635682b6849b3a2a890e56fb7a8e
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
Mounted at /content/drive
'Code=005930'  'Code=035420'  'Code=064260'  'Code=068270'
